## Loading all Data

In [23]:
import os
import json
import pandas as pd
from glob import glob
import numpy

# Define the base directories
base_dirs = ['train', 'eval', 'test']

for i in range(len(base_dirs)):
    base_dirs[i] = '../../../data/methods2test_data/' + base_dirs[i]
# List to collect all data (rows)
data = []

#total_exmaples = 0 FOR BREAKING**

#is_broken = False FOR BREAKING**
for base_dir in base_dirs:
    # Iterate over all subdirectories (examples) in the base directory
    for example_dir in os.listdir(base_dir):
        example_path = os.path.join(base_dir, example_dir)
        
        # Ensure we're dealing with directories (i.e., each example is its own directory)
        if os.path.isdir(example_path):
            # Use glob to get all JSON files in the example directory
            json_files = glob(os.path.join(example_path, '*.json'))
            
            # Iterate over the JSON files
            for json_file in json_files:
                # Read the JSON data
                with open(json_file, 'r') as f:
                    json_data = json.load(f)
    
                #Add original file location to data set (TRY TO SHORTEN TO ONLY EVAL-TRAIN-TEST/DIRECTORY NUMBER/JSON FILE NAME)
                json_data['original_file'] = json_file[32:] #This is currently the full path to the file
                
                # Add the JSON data to the data list (can modify if additional info is needed)
                data.append(json_data)
                #total_exmaples+=1 FOR BREAKING
                #if(total_exmaples>=100):
                    #print("breaking")
                    #is_broken = True
                    #break
        #if(is_broken):
            #break END OF FOR BREAKING
# Convert the collected data into a pandas DataFrame
df = pd.DataFrame(data)

# Optional: Inspect the first few rows
#print(df.head())

In [15]:
def clean_file_path(filepath):
    new_filepath = filepath[32:]
    return new_filepath
clean_file_path('../../../data/methods2test_data/train')

'train'

In [7]:

with open("../../../data/methods2test_data/eval/81160/81160_0.json", 'r') as f:
    json_data = json.load(f)
json_data['original_file'] = "../../../data/methods2test_data/eval/81160/81160_0.json"

In [10]:
json_data.keys()

dict_keys(['test_class', 'test_case', 'focal_class', 'focal_method', 'repository', 'original_file'])

In [25]:
df.to_csv('../../../data/all_examples.csv')

In [ ]:
df = pd.read_csv("../../../data/all_examples.csv")

## Full List/Examples of Dataset Attributes

In [24]:
df.iloc[7]['focal_method']

{'identifier': 'setProperty',
 'parameters': '(String name, boolean value)',
 'modifiers': '@Override public',
 'return': 'JMSProducer',
 'body': '@Override\n    public JMSProducer setProperty(String name, boolean value) {\n        return setObjectProperty(name, value);\n    }',
 'signature': 'JMSProducer setProperty(String name, boolean value)',
 'full_signature': '@Override public JMSProducer setProperty(String name, boolean value)',
 'class_method_signature': 'JmsPoolJMSProducer.setProperty(String name, boolean value)',
 'testcase': False,
 'constructor': False,
 'invocations': ['setObjectProperty']}

In [25]:
df.iloc[7]['focal_class']

{'identifier': 'JmsPoolJMSProducer',
 'superclass': '',
 'interfaces': 'implements JMSProducer',
 'fields': [{'original_string': 'private final JmsPoolSession session;',
   'modifier': 'private final',
   'type': 'JmsPoolSession',
   'declarator': 'session',
   'var_name': 'session'},
  {'original_string': 'private final JmsPoolMessageProducer producer;',
   'modifier': 'private final',
   'type': 'JmsPoolMessageProducer',
   'declarator': 'producer',
   'var_name': 'producer'},
  {'original_string': 'private CompletionListener completionListener;',
   'modifier': 'private',
   'type': 'CompletionListener',
   'declarator': 'completionListener',
   'var_name': 'completionListener'},
  {'original_string': 'private String correlationId;',
   'modifier': 'private',
   'type': 'String',
   'declarator': 'correlationId',
   'var_name': 'correlationId'},
  {'original_string': 'private String type;',
   'modifier': 'private',
   'type': 'String',
   'declarator': 'type',
   'var_name': 'type'

In [26]:
df.iloc[4]['test_case']

{'identifier': 'testCheckValidObjectList',
 'parameters': '()',
 'modifiers': '@Test(expected = MessageFormatException.class) public',
 'return': 'void',
 'body': '@Test(expected = MessageFormatException.class)\n    public void testCheckValidObjectList() throws JMSException {\n        JMSMessagePropertySupport.checkValidObject(Collections.EMPTY_LIST);\n    }',
 'signature': 'void testCheckValidObjectList()',
 'full_signature': '@Test(expected = MessageFormatException.class) public void testCheckValidObjectList()',
 'class_method_signature': 'JMSMessagePropertySupportTest.testCheckValidObjectList()',
 'testcase': True,
 'constructor': False,
 'invocations': ['checkValidObject']}

In [27]:
df.iloc[7]['test_class']

{'identifier': 'JmsPoolJMSProducerTest',
 'superclass': 'extends JmsPoolTestSupport',
 'interfaces': '',
 'fields': [{'original_string': 'private final String STRING_PROPERTY_NAME = "StringProperty";',
   'modifier': 'private final',
   'type': 'String',
   'declarator': 'STRING_PROPERTY_NAME = "StringProperty"',
   'var_name': 'STRING_PROPERTY_NAME'},
  {'original_string': 'private final String STRING_PROPERTY_VALUE = UUID.randomUUID().toString();',
   'modifier': 'private final',
   'type': 'String',
   'declarator': 'STRING_PROPERTY_VALUE = UUID.randomUUID().toString()',
   'var_name': 'STRING_PROPERTY_VALUE'},
  {'original_string': 'private final String BYTE_PROPERTY_NAME = "ByteProperty";',
   'modifier': 'private final',
   'type': 'String',
   'declarator': 'BYTE_PROPERTY_NAME = "ByteProperty"',
   'var_name': 'BYTE_PROPERTY_NAME'},
  {'original_string': 'private final byte BYTE_PROPERTY_VALUE = Byte.MAX_VALUE;',
   'modifier': 'private final',
   'type': 'byte',
   'declarator'

## Organizing Prompt

In [26]:
import re
'''
This function takes each example and crafts the prompt
that will be fed to the models.

The function takes the full test method body and captures
the full signature and modifiers in correct format up
until the beginning of the actual test code (first open
parenthesis "{").
'''
def create_prompts(df):
    test_prompt = ""
    prompt_list = []
    failed_indicies = []
    for index, row in df.iterrows():
        test_method_name = row['test_case']['identifier']
        test_code = row['test_case']['body']

        pattern = re.escape(test_method_name) + r'.*?\s\{'
        match = re.search(pattern, test_code)
        if not match:
            failed_indicies.append({"failed_indicies": index, "original_file": row['original_file']})
            continue

        focal_method = df.iloc[i]['focal_method']['body']
        focal_method_name = df.iloc[4]['focal_method']['identifier']
        
        test_prompt = f'''
{focal_method}

//Create unit tests for the {focal_method_name} method given above
        
{test_code[:match.end()]}
        '''

        prompt_list.append({"original_index": index, "prompt": test_prompt, "original_file": row['original_file']})
    prompt_df = pd.DataFrame(prompt_list)
    prompt_df.to_csv("../../../data/prompts/first_formatted_prompts.csv")

    failed_indicies_df = pd.DataFrame(failed_indicies)
    failed_indicies_df.to_csv("../../../data/prompts/failed_indicies.csv")
    return prompt_df, failed_indicies_df

new_prompts, failed = create_prompts(df=df)

print(new_prompts.iloc[7]['prompt'])


@Override
	public void removeAll(Iterator<? extends Statement> iterator) throws ModelRuntimeException {
		if(this.isLocked()) {
			throw new ModelRuntimeException("Model is locked, cannot perform an update.");
		}
		// do not auto-commit
		assertModel();
		try {
			this.connection.begin();
			try {
				try {
					// remove all
					while(iterator.hasNext()) {
						org.openrdf.model.Statement s = ConversionUtil.toOpenRDF(iterator.next(),
						        this.valueFactory);
						this.connection.remove(s, this.openRdfContext);
					}
					this.connection.commit();
				} catch(RepositoryException x) {
					this.connection.rollback();
				}
			} finally {
				this.connection.commit();
			}
		} catch(RepositoryException x) {
			throw new ModelRuntimeException(x);
		}
	}

//Create unit tests for the checkValidObject method given above
        
@Test
    public void testSetObjectPropertyWithInvalidObject() {
        


In [27]:
len(new_prompts)

708093

In [29]:
new_prompts.iloc[0]

original_index                                                    0
prompt            \n@Override\n\tpublic void removeAll(Iterator<...
original_file                        train/32802417/32802417_2.json
Name: 0, dtype: object

In [ ]:
del(df)

In [43]:
print(df.iloc[7]["test_case"]["body"])

@Test
    public void testSetObjectPropertyWithInvalidObject() {
        JMSProducer producer = context.createProducer();

        try {
            producer.setProperty(GOOD_PROPERTY_NAME, UUID.randomUUID());
            fail("Should not accept invalid property name");
        } catch (MessageFormatRuntimeException mfre) {}
    }
